# Applied Data Science Capstone

This is the notebook for the Coursera course 'Applied Data Science Capstone'

In [7]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [8]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [9]:
# Download url and save to variable 'url'
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data  = requests.get(url).text 

In [10]:
soup = BeautifulSoup(data,"html5lib")  # create a soup object using the variable 'data'

In [51]:
# Find tbody inside table
tbody = soup.find_all('tbody')

In [57]:
# Take data from the table and put it into dataframe called 'toronto_df'
toronto_df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])

count = 1
for table in tbody:
    if count == 1:
        for row in table.find_all('tr')[1:]:
            cols = row.find_all('td')
            postal_code = str(cols[0])[4:-6]
            borough = str(cols[1])[4:-6]
            neighborhood = str(cols[2])[4:-6]
            
            toronto_df = toronto_df.append({'PostalCode' : postal_code, 'Borough' : borough, 'Neighborhood' : neighborhood }, ignore_index = True)

    count = 0

In [58]:
#remove rows that have 'Not assigned' values in the column 'Borough', then reset index.
toronto_df = toronto_df.query('Borough != "Not assigned"')
toronto_df.reset_index(inplace = True, drop = True)

In [61]:
toronto_df.shape

(103, 3)